# Current Todo:


Some ideas here: 

* Use pandas_ta to attach technical data to timeseries then use random forest to see if any features are super good.

* Use vector_bt to back test your models

* Integrate the different time granularity models, 15m etc into the core configs

* Fix coinbase - we can't filter on volume amount without extra work

* Add TA fields, use random forest to check which fields are the best, verify with mean square error (this could take a while)

* Move to an "always" on model where we more closely monitor exits and entries. We can constantly check predictions and refine entry/exit points. Can we pick a better entry point? We had at some point discussed trying to predict all 4 values which would help. 

* Can we monitor and alarm/email the 15 min chart of an entry and see if it is
move toward or away from our target?

* pandas_ta strategy/back testers look interesting?

Read me: https://www.kaggle.com/code/vuhuyduongnia/vn30-stock-prediction-by-lstm-model-accuracy-90


#IMPORT DATASETS AND LIBRARIES


In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import sys

sys.path.insert(0,'/content/drive/My Drive/ml-trde-notebooks')


import warnings
import pandas as pd

# suppress warning message
warnings.filterwarnings("ignore", message="The frame.append method is deprecated")


Mounted at /content/drive


# Library

In [2]:
# ***** WARNING : Install deps - This will BUILD TALib and takes a while!
%run -i '/content/drive/My Drive/ml-trde-notebooks/installs.ipynb'


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218921 sha256=043f57874b3537a94aaee065cc30f616620285a0511f530fbb19407425d571ad
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 5.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking

In [3]:
import yfinance as yf
import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose
import plotly.graph_objs as go
import numpy as np
from scipy.stats import norm
from scipy.signal import find_peaks, peak_prominences

In [4]:
# Executes this notebook in our space, making all of its functions/globals available
%run -i '/content/drive/My Drive/ml-trde-notebooks/common.ipynb'
train_models = False
save_models = False
load_models = True
if load_models:
  load_all_models()
else:
  print ("Not loading")

Modified
1682589663
1682589663.0
1682589663.0
loading: lstm_cv
models loaded


# What has a buy indicator for tomorrow?

In [5]:
# Fetch the top 10 and see if they predict up
coin_base = False
ku_coin = True
df_products = get_all_products()
df_products = df_products[df_products.id.str.endswith('USDT')]

report_columns = ["Product",
                  "Model Name",
                  "Close",
                  "Target",
                  "Predicted",
                  "MSE",
                  "Percent",
                  "250Fees",
                  "5kFees",
                  "10kFees",
                  "250Profit",
                  "5kProfit",
                  "10k0Profit",
                  "Period"]

# we have a desired budget of 10k in trading capital to deploy. 
# "volValue": is the 24h total, the trading volume in quote currency of last 24 hours
# Any markets we enter need to have a signifcantly higher trading value volume otherwise
# we can't really trade there without breaking things up. As we grow our strat here will need to change.
# for now we cut down our set to 1m in volVal
if ku_coin:
  volumeCutoff = 1000000
  df_products["volValue"] = [float(x) for x in df_products['volValue']]
  df_products = df_products[df_products["volValue"] > volumeCutoff]
  df_products = df_products.sort_values(by = ['id'])

models_in_play = ["lstm_cv"]

if coin_base:
  df_products = df_products[df_products.trading_disabled == False]
  df_products = df_products[df_products.cancel_only == False]

df_trades = pd.DataFrame();
df_estc = pd.DataFrame(); #expected short term closes
df_estc["Product"] = [];
df_estc["Est Close"] = [];
df_estc["Est Close Raw"] = [];

bars_long = 300

bars_short = 30
counter = 0;
print("iterating through:", len(df_products))
tries = 3

def downloadAndPredict(all_trades, product, length):
  print("download day bars: ", product, " bar set:", length)
  df_raw = get_coin_data_frames(length, product)
  for name in models_in_play:
    print("predicting trade:", name, "for", product)
    df_trade = predict_config_model_for_product(df_raw, name, product)
    df_trade["Period"] = [length]
    df_trade = df_trade[report_columns]
    trend, prob_above, prob_below, volatility = generate_day_probability(df_raw)
    df_trade["trend"] = [trend]
    df_trade["prob_above"] = [prob_above]
    df_trade["prob_below"] = [prob_below]
    df_trade["volatility"] = [volatility]
    print("predicted mse:", name, ":{:.10f}".format(df_trade["MSE"].iloc[0]))
    all_trades = all_trades.append(df_trade)
  return all_trades

def downloadAndPredict15(all_trades, product):
  print("download 15 bars: ", product)
  [predicted_scaled, predicted] = fetch_and_predict_short_term(lstm_15m,product)
  df2 = pd.DataFrame({'Product': [product], 'Est Close': [predicted], 'Est Close Raw': predicted_scaled})
  all_trades = all_trades.append(df2)
  return all_trades


for index, row in df_products.iterrows():
    
    loop = True
    count = 0
    while(loop):
      try:
        print("start day long")
        df_trades = downloadAndPredict(df_trades, row.id, bars_long)
        time.sleep(1)
        
        #print("start day short")
        #df_trades = downloadAndPredict(df_trades, row.id, bars_short)
        #time.sleep(1)

        #print("start 15m")
        #df_estc = downloadAndPredict15(df_estc, row.id)
        #time.sleep(1)
        loop = False
      except Exception as inst:
        #raise inst
        print("Error: ", inst)
        time.sleep(1)
        count = count+1
        if count>tries:
          loop = False

df_trades.reset_index()
df_trades_filtered = df_trades  # TODO: Filter again after see what the new mse's look like #df_trades.loc[(df_trades["Period"] == 180) & (df_trades["MSE"] < 0.00005) | (df_trades["Period"] == 30) & (df_trades["MSE"] < 0.0005)]
df_buys = df_trades_filtered[df_trades_filtered['Percent'] > 0] 
df_shorts = df_trades_filtered[df_trades_filtered['Percent'] < 0] 
df_weighted = df_trades_filtered.groupby("Product").apply(consensus_prediction)
df_weighted = df_weighted.rename("Consensus Prediction")
df_trades_final = pd.merge(df_trades_filtered, df_weighted, left_on='Product', right_index=True)
df_trades_final["Consensus Percent"] = ((df_trades_final['Consensus Prediction'] - df_trades_final['Close']) / df_trades_final['Close']) * 100
df_view = df_trades_final[["Product",  "Model Name",  "MSE", "Period", "Close", "Predicted", "Percent","trend", "prob_above", "prob_below", "volatility"]]
if coin_base:
  coin_base_view = df_view
else:
  ku_coin_view = df_view

iterating through: 67
start day long
download day bars:  ACOIN-USDT  bar set: 300
predicting trade: lstm_cv for ACOIN-USDT
predicted mse: lstm_cv :0.0000191094
start day long
download day bars:  ADA-USDT  bar set: 300
predicting trade: lstm_cv for ADA-USDT
predicted mse: lstm_cv :0.0001195859
start day long
download day bars:  AGIX-USDT  bar set: 300
predicting trade: lstm_cv for AGIX-USDT
predicted mse: lstm_cv :0.0000447877
start day long
download day bars:  ALBT-USDT  bar set: 300
Error:  Illegal response: {"msg":"This pair is not provided at present.","code":"400100"}
start day long
download day bars:  ALBT-USDT  bar set: 300
Error:  Illegal response: {"msg":"This pair is not provided at present.","code":"400100"}
start day long
download day bars:  ALBT-USDT  bar set: 300
Error:  Illegal response: {"msg":"This pair is not provided at present.","code":"400100"}
start day long
download day bars:  ALBT-USDT  bar set: 300
Error:  Illegal response: {"msg":"This pair is not provided at p

In [6]:
now = datetime.now()
today = now.strftime("%Y-%m-%d")

if coin_base:
  token = "cb"
else:
  token = "ku"

#df_buys.to_csv(data_path+"/buy-" + token + "-" + today + ".csv")
#df_estc.to_csv(data_path+"/15m-" + token + "-" + today + ".csv")
#df_shorts.to_csv(data_path+"/shorts-" + token + "-" + today + ".csv")
#df_trades_final.to_csv(data_path+"/final-" + token + "-" + today + ".csv")
df_view.to_csv(data_path+"/view-" + token + "-" + today + ".csv")

In [7]:
df_view

,Product,Model Name,MSE,Period,Close,Predicted,Percent,trend,prob_above,prob_below,volatility
Date,,,,,,,,,,,
2023-04-28,ACOIN-USDT,lstm_cv,0.000019,300,0.056379,0.056594,0.381280,0.025757,0.880239,0.119761,-0.029253
2023-04-28,ADA-USDT,lstm_cv,0.000120,300,0.405867,0.403619,-0.553868,0.568577,0.986457,0.013543,-0.028912
2023-04-28,AGIX-USDT,lstm_cv,0.000045,300,0.341630,0.340386,-0.364172,0.046126,0.356249,0.643751,-0.044877
2023-04-28,ALGO-USDT,lstm_cv,0.000048,300,0.183000,0.184299,0.709645,0.362261,0.915780,0.084220,-0.030663
2023-04-28,ATOM-USDT,lstm_cv,0.000137,300,11.403700,11.225028,-1.566789,12.909916,0.981750,0.018250,-0.075885
2023-04-28,AVAX-USDT,lstm_cv,0.000108,300,17.592000,17.482979,-0.619718,25.883205,0.999495,0.000505,-0.071074
2023-04-28,BNB-USDT,lstm_cv,0.000098,300,320.757000,319.484708,-0.396653,284.370766,0.993544,0.006456,-0.053360
2023-04-28,BTC-USDT,lstm_cv,0.000104,300,29191.200000,29130.756200,-0.207062,23465.529567,0.989818,0.010182,-0.045439
2023-04-28,CFX-USDT,lstm_cv,0.000060,300,0.327765,0.328223,0.139659,0.065187,0.846145,0.153855,-0.033741
